In [1]:
import sys, os, time
import logging
import torch
import pickle
import numpy as np
import cv2
sys.path.insert(0, '/home/rajeev-gupta/sensyn_ws/src/GD-MAE')


In [2]:
from pcdet.models.detectors import GraphRCNN
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets.kitti.kitti_dataset import KittiDataset


In [3]:
log_path = './test_logs.txt'
cfg_file = '/home/rajeev-gupta/sensyn_ws/src/GD-MAE/tools/cfgs/kitti_models/graph_rcnn_voi.yaml'
ckpt_path = '/home/rajeev-gupta/sensyn_ws/src/GD-MAE/data/ckpts/graph_rcnn_voi_kitti.pth'
to_cpu = False

In [4]:
#Logger
def create_logger(log_file=None, rank=0, log_level=logging.INFO):
    logger = logging.getLogger(__name__)
    logger.setLevel(log_level if rank == 0 else 'ERROR')
    formatter = logging.Formatter('%(asctime)s  %(levelname)5s  %(message)s')
    console = logging.StreamHandler()
    console.setLevel(log_level if rank == 0 else 'ERROR')
    console.setFormatter(formatter)
    logger.addHandler(console)
    if log_file is not None:
        file_handler = logging.FileHandler(filename=log_file)
        file_handler.setLevel(log_level if rank == 0 else 'ERROR')
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
    logger.propagate = False
    return logger

In [5]:
def load_data_to_gpu(batch_dict):
    for key, val in batch_dict.items():
        if not isinstance(val, np.ndarray):
            continue
        elif key in ['frame_id', 'metadata', 'calib', 'image_shape', 'image_pad_shape', 'image_rescale_shape']:
            continue
        else:
            batch_dict[key] = torch.from_numpy(val).float().cuda()


In [6]:
logger = create_logger(log_path)

In [7]:
cfg_from_yaml_file(cfg_file, cfg)

{'ROOT_DIR': PosixPath('/home/rajeev-gupta/sensyn_ws/src/GD-MAE'),
 'LOCAL_RANK': 0,
 'CLASS_NAMES': ['Car'],
 'DATA_CONFIG': {'DATASET': 'KittiDataset',
  'DATA_PATH': '../data/kitti',
  'BACKEND': {'NAME': 'HardDiskBackend'},
  'POINT_CLOUD_RANGE': [0, -40, -3, 70.4, 40, 1],
  'DATA_SPLIT': {'train': 'train', 'test': 'test'},
  'INFO_PATH': {'train': ['kitti_infos_train.pkl'],
   'test': ['kitti_infos_test.pkl']},
  'GET_ITEM_LIST': ['points', 'image', 'calib_matricies', 'gt_boxes2d'],
  'FOV_POINTS_ONLY': True,
  'ENABLE_SIMILAR_TYPE': True,
  'DATA_AUGMENTOR': {'DISABLE_AUG_LIST': ['placeholder'],
   'AUG_CONFIG_LIST': [{'NAME': 'random_world_flip',
     'PROBABILITY': 0.5,
     'ALONG_AXIS_LIST': ['x']},
    {'NAME': 'random_world_rotation',
     'PROBABILITY': 1.0,
     'WORLD_ROT_ANGLE': [-0.78539816, 0.78539816]},
    {'NAME': 'random_world_scaling',
     'PROBABILITY': 1.0,
     'WORLD_SCALE_RANGE': [0.95, 1.05]}]},
  'POINT_FEATURE_ENCODING': {'encoding_type': 'absolute_coord

In [8]:
def get_calib_from_file(calib_file):
    lines = open(calib_file).readlines()

    obj = lines[2].strip().split(' ')[1:]
    P2 = np.array(obj, dtype=np.float32)
    obj = lines[3].strip().split(' ')[1:]
    P3 = np.array(obj, dtype=np.float32)
    obj = lines[4].strip().split(' ')[1:]
    R0 = np.array(obj, dtype=np.float32)
    obj = lines[5].strip().split(' ')[1:]
    Tr_velo_to_cam = np.array(obj, dtype=np.float32)

    return {'P2': P2.reshape(3, 4),
            'P3': P3.reshape(3, 4),
            'R0': R0.reshape(3, 3),
            'Tr_velo2cam': Tr_velo_to_cam.reshape(3, 4)}

def calib_to_matricies(calib):
    """
    Converts calibration object to transformation matricies
    Args:
        calib: calibration.Calibration, Calibration object
    Returns
        V2R: (4, 4), Lidar to rectified camera transformation matrix
        P2: (3, 4), Camera projection matrix
    """
    V2C = np.vstack((calib.V2C, np.array([0, 0, 0, 1], dtype=np.float32)))  # (4, 4)
    R0 = np.hstack((calib.R0, np.zeros((3, 1), dtype=np.float32)))  # (3, 4)
    R0 = np.vstack((R0, np.array([0, 0, 0, 1], dtype=np.float32)))  # (4, 4)
    V2R = R0 @ V2C
    P2 = calib.P2
    return V2R, P2

In [9]:
class Calibration(object):
    def __init__(self, calib_file):
        if not isinstance(calib_file, dict):
            calib = get_calib_from_file(calib_file)
        else:
            calib = calib_file

        self.P2 = calib['P2']  # 3 x 4
        self.R0 = calib['R0']  # 3 x 3
        self.V2C = calib['Tr_velo2cam']  # 3 x 4

        # Camera intrinsics and extrinsics
        self.cu = self.P2[0, 2]
        self.cv = self.P2[1, 2]
        self.fu = self.P2[0, 0]
        self.fv = self.P2[1, 1]
        self.tx = self.P2[0, 3] / (-self.fu)
        self.ty = self.P2[1, 3] / (-self.fv)

    def cart_to_hom(self, pts):
        """
        :param pts: (N, 3 or 2)
        :return pts_hom: (N, 4 or 3)
        """
        pts_hom = np.hstack((pts, np.ones((pts.shape[0], 1), dtype=np.float32)))
        return pts_hom

    def rect_to_lidar(self, pts_rect):
        """
        :param pts_lidar: (N, 3)
        :return pts_rect: (N, 3)
        """
        pts_rect_hom = self.cart_to_hom(pts_rect)  # (N, 4)
        R0_ext = np.hstack((self.R0, np.zeros((3, 1), dtype=np.float32)))  # (3, 4)
        R0_ext = np.vstack((R0_ext, np.zeros((1, 4), dtype=np.float32)))  # (4, 4)
        R0_ext[3, 3] = 1
        V2C_ext = np.vstack((self.V2C, np.zeros((1, 4), dtype=np.float32)))  # (4, 4)
        V2C_ext[3, 3] = 1

        pts_lidar = np.dot(pts_rect_hom, np.linalg.inv(np.dot(R0_ext, V2C_ext).T))
        return pts_lidar[:, 0:3]

    def lidar_to_rect(self, pts_lidar):
        """
        :param pts_lidar: (N, 3)
        :return pts_rect: (N, 3)
        """
        pts_lidar_hom = self.cart_to_hom(pts_lidar)
        pts_rect = np.dot(pts_lidar_hom, np.dot(self.V2C.T, self.R0.T))
        # pts_rect = reduce(np.dot, (pts_lidar_hom, self.V2C.T, self.R0.T))
        return pts_rect

    def rect_to_img(self, pts_rect):
        """
        :param pts_rect: (N, 3)
        :return pts_img: (N, 2)
        """
        pts_rect_hom = self.cart_to_hom(pts_rect)
        pts_2d_hom = np.dot(pts_rect_hom, self.P2.T)
        pts_img = (pts_2d_hom[:, 0:2].T / pts_rect_hom[:, 2]).T  # (N, 2)
        pts_rect_depth = pts_2d_hom[:, 2] - self.P2.T[3, 2]  # depth in rect camera coord
        return pts_img, pts_rect_depth

    def lidar_to_img(self, pts_lidar):
        """
        :param pts_lidar: (N, 3)
        :return pts_img: (N, 2)
        """
        pts_rect = self.lidar_to_rect(pts_lidar)
        pts_img, pts_depth = self.rect_to_img(pts_rect)
        return pts_img, pts_depth

    def img_to_rect(self, u, v, depth_rect):
        """
        :param u: (N)
        :param v: (N)
        :param depth_rect: (N)
        :return:
        """
        x = ((u - self.cu) * depth_rect) / self.fu + self.tx
        y = ((v - self.cv) * depth_rect) / self.fv + self.ty
        pts_rect = np.concatenate((x.reshape(-1, 1), y.reshape(-1, 1), depth_rect.reshape(-1, 1)), axis=1)
        return pts_rect

    def corners3d_to_img_boxes(self, corners3d):
        """
        :param corners3d: (N, 8, 3) corners in rect coordinate
        :return: boxes: (None, 4) [x1, y1, x2, y2] in rgb coordinate
        :return: boxes_corner: (None, 8) [xi, yi] in rgb coordinate
        """
        sample_num = corners3d.shape[0]
        corners3d_hom = np.concatenate((corners3d, np.ones((sample_num, 8, 1))), axis=2)  # (N, 8, 4)

        img_pts = np.matmul(corners3d_hom, self.P2.T)  # (N, 8, 3)

        x, y = img_pts[:, :, 0] / img_pts[:, :, 2], img_pts[:, :, 1] / img_pts[:, :, 2]
        x1, y1 = np.min(x, axis=1), np.min(y, axis=1)
        x2, y2 = np.max(x, axis=1), np.max(y, axis=1)

        boxes = np.concatenate((x1.reshape(-1, 1), y1.reshape(-1, 1), x2.reshape(-1, 1), y2.reshape(-1, 1)), axis=1)
        boxes_corner = np.concatenate((x.reshape(-1, 8, 1), y.reshape(-1, 8, 1)), axis=2)

        return boxes, boxes_corner


In [10]:
from pathlib import Path

import numpy as np
import torch.utils.data as torch_data

# from ..utils import common_utils, file_client
# from ..pcdet.datasets.augmentor.data_augmentor import DataAugmentor
from pcdet.datasets.processor.data_processor import DataProcessor
from pcdet.datasets.processor.point_feature_encoder import PointFeatureEncoder

class CustomKittiDataset(torch_data.Dataset):
    def __init__(self, dataset_cfg=None, class_names=None, training=True, root_path=None, logger=None):
        super().__init__()
        self.dataset_cfg = dataset_cfg
        self.training = training
        self.class_names = class_names
        self.logger = logger
        self.root_path = Path(root_path) if root_path is not None else Path(self.dataset_cfg.DATA_PATH)

        self.point_cloud_range = np.array(self.dataset_cfg.POINT_CLOUD_RANGE, dtype=np.float32)
        self.point_feature_encoder = PointFeatureEncoder(
            self.dataset_cfg.POINT_FEATURE_ENCODING,
            point_cloud_range=self.point_cloud_range
        )
        self.data_processor = DataProcessor(
            self.dataset_cfg.DATA_PROCESSOR, point_cloud_range=self.point_cloud_range,
            training=self.training, num_point_features=self.point_feature_encoder.num_point_features
        )
        self.grid_size = self.data_processor.grid_size
        self.voxel_size = self.data_processor.voxel_size
        self.total_epochs = 0
        self.cur_epoch = 0
        self._merge_all_iters_to_one_epoch = False

    @property
    def mode(self):
        return 'train' if self.training else 'test'
        
    def __getitem__(self, index):
        info_path = '/media/rajeev-gupta/Drive250/data/kitti/kitti_infos_test.pkl'
        with open(info_path, 'rb') as i_file:
            i_dict = pickle.load(i_file)
        info = i_dict[index]
        
        sample_idx = info['point_cloud']['lidar_idx']
        img_shape = info['image']['image_shape']
        calib = self.get_calib(sample_idx)
        get_item_list = self.dataset_cfg.get('GET_ITEM_LIST', ['points'])

        input_dict = {
            'frame_id': sample_idx,
            'calib': calib,
        }

        if "points" in get_item_list:
            points = self.get_lidar(sample_idx)
            if self.dataset_cfg.FOV_POINTS_ONLY:
                pts_rect = calib.lidar_to_rect(points[:, 0:3])
                fov_flag = self.get_fov_flag(pts_rect, img_shape, calib)
                points = points[fov_flag]
            input_dict['points'] = points

        if "image" in get_item_list:
            input_dict['image'] = self.get_image(sample_idx)

        if "calib_matricies" in get_item_list:
            input_dict["trans_lidar_to_cam"], input_dict["trans_cam_to_img"] = calib_to_matricies(calib)

        data_dict = self.prepare_data(data_dict=input_dict)

        data_dict['image_shape'] = img_shape
        return data_dict
    
    def get_calib(self, idx):
        calib_file = self.root_path / 'testing' / 'calib' / ('%s.txt' % idx)
        return Calibration(calib_file)

    def get_lidar(self, idx):
        lidar_file = self.root_path / 'testing' / 'velodyne' / ('%s.bin' % idx)
        return np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 4)
    
    def get_image(self, idx):
        img_file = self.root_path / 'testing' / 'image_2' / ('%s.png' % idx)
        return cv2.imread(str(img_file), cv2.IMREAD_COLOR)
    
    def prepare_data(self, data_dict):
        if data_dict.get('points', None) is not None:
            data_dict = self.point_feature_encoder.forward(data_dict)

        data_dict = self.data_processor.forward(
            data_dict=data_dict
        )
        return data_dict

    @staticmethod
    def get_fov_flag(pts_rect, img_shape, calib):
        print('def get_fov_flag ******************')
        """
        Args:
            pts_rect:
            img_shape:
            calib:

        Returns:

        """
        pts_img, pts_rect_depth = calib.rect_to_img(pts_rect)
        val_flag_1 = np.logical_and(pts_img[:, 0] >= 0, pts_img[:, 0] < img_shape[1])
        val_flag_2 = np.logical_and(pts_img[:, 1] >= 0, pts_img[:, 1] < img_shape[0])
        val_flag_merge = np.logical_and(val_flag_1, val_flag_2)
        pts_valid_flag = np.logical_and(val_flag_merge, pts_rect_depth >= 0)

        return pts_valid_flag


In [11]:
dataset = CustomKittiDataset(
        dataset_cfg=cfg.DATA_CONFIG,
        class_names=cfg.CLASS_NAMES,
        root_path='/home/rajeev-gupta/sensyn_ws/src/GD-MAE/data/kitti',
        training=False,
        logger=logger,
    )

In [12]:
# dataset = KittiDataset(
#         dataset_cfg=cfg.DATA_CONFIG,
#         class_names=cfg.CLASS_NAMES,
#         root_path=None,
#         training=False,
#         logger=logger,
#     )

In [13]:
model = GraphRCNN(cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=dataset, logger=logger)

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [ ]:
with open('/home/rajeev-gupta/sensyn_ws/src/GD-MAE/tools/batch_dict_22', 'rb') as b_file:
    b_dict = pickle.load(b_file)
print(type(b_dict))
b_dict

<class 'dict'>


{'frame_id': array(['000022'], dtype='<U6'),
 'calib': array([<pcdet.utils.calibration_kitti.Calibration object at 0x7f1f98d70e10>],
       dtype=object),
 'points': array([[ 0.0000e+00,  5.6430e+01,  1.1790e+00,  2.1180e+00,  0.0000e+00],
        [ 0.0000e+00,  6.2003e+01,  1.4930e+00,  2.3060e+00,  0.0000e+00],
        [ 0.0000e+00,  4.7137e+01,  6.2750e+00,  1.8170e+00,  9.2000e-01],
        ...,
        [ 0.0000e+00,  6.3400e+00, -4.1000e-02, -1.6560e+00,  2.7000e-01],
        [ 0.0000e+00,  6.3420e+00, -3.1000e-02, -1.6560e+00,  2.3000e-01],
        [ 0.0000e+00,  6.3380e+00, -1.1000e-02, -1.6550e+00,  1.8000e-01]],
       dtype=float32),
 'image': array([[[[ 5.4155402e-02,  1.4589565e-01,  1.8761617e-01, ...,
            0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
          [-1.2936717e-01,  3.0806570e-04,  4.3419853e-02, ...,
            0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
          [-1.7442521e-01, -4.3367714e-02, -2.0523893e-04, ...,
            0.0000000e+00, 

In [ ]:
get_dict = dataset.__getitem__(4)
get_dict

def get_fov_flag ******************
point is not None
from def forward ******************
from def mask_points_and_boxes_outside_range ******************
from def shuffle_points ******************
from def calculate_grid_size ******************
from def imrescale ******************
from def imflip ******************
from def imnormalize ******************
from def impad ******************


{'frame_id': '000004',
 'calib': <__main__.Calibration at 0x7f1f98d6ed90>,
 'points': array([[ 3.7786e+01,  7.8970e+00,  1.5140e+00,  9.4000e-01],
        [ 3.7753e+01,  8.0140e+00,  1.5140e+00,  9.9000e-01],
        [ 3.7741e+01,  8.1360e+00,  1.5150e+00,  9.9000e-01],
        ...,
        [ 6.3800e+00, -3.2000e-02, -1.6670e+00,  2.2000e-01],
        [ 6.3980e+00, -2.2000e-02, -1.6720e+00,  2.1000e-01],
        [ 6.3770e+00, -1.0000e-03, -1.6660e+00,  1.2000e-01]],
       dtype=float32),
 'image': array([[[-1.277312  , -1.5028257 , -1.5781852 ],
         [-1.277312  , -1.5028257 , -1.5781852 ],
         [-1.2908984 , -1.4599018 , -1.5781852 ],
         ...,
         [ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]],
 
        [[-1.3044848 , -1.4885178 , -1.5781852 ],
         [-1.277312  , -1.5028257 , -1.5781852 ],
         [-1.277312  , -1.4885178 , -1.5922866 ],
         ...,
         [ 0.   

In [ ]:
for key, val in get_dict.items():
    print(key)
    if type(val) == tuple:
        get_dict[key] = list(val)
    elif key == 'points':
        # add a zero column
        n = val.shape[0]
        z_col = np.zeros((n, 1), dtype=float)
        get_dict[key] = np.concatenate((z_col, val), axis = 1)
        continue
    elif key == 'image':
        # transpose (384, 1280, 3) to (3, 384, 1280)
        val_transposed = np.transpose(val, (2, 0, 1))
        get_dict[key] = val_transposed
        # print(get_dict[key].shape)
    elif key == 'transformation_2d_list' or key == 'transformation_2d_params':
        get_dict[key] = [val]
        continue
    get_dict[key] = np.array([get_dict[key]])
            
get_dict['batch_size'] = 1



frame_id
calib
points
image
trans_lidar_to_cam
trans_cam_to_img
use_lead_xyz
transformation_2d_list
transformation_2d_params
image_rescale_shape
image_pad_shape
image_shape


In [ ]:
get_dict

{'frame_id': array(['000004'], dtype='<U6'),
 'calib': array([<__main__.Calibration object at 0x7f1f98d6ed90>], dtype=object),
 'points': array([[ 0.00000000e+00,  3.77859993e+01,  7.89699984e+00,
          1.51400006e+00,  9.39999998e-01],
        [ 0.00000000e+00,  3.77529984e+01,  8.01399994e+00,
          1.51400006e+00,  9.90000010e-01],
        [ 0.00000000e+00,  3.77410011e+01,  8.13599968e+00,
          1.51499999e+00,  9.90000010e-01],
        ...,
        [ 0.00000000e+00,  6.38000011e+00, -3.20000015e-02,
         -1.66700006e+00,  2.19999999e-01],
        [ 0.00000000e+00,  6.39799976e+00, -2.19999999e-02,
         -1.67200005e+00,  2.09999993e-01],
        [ 0.00000000e+00,  6.37699986e+00, -1.00000005e-03,
         -1.66600001e+00,  1.19999997e-01]]),
 'image': array([[[[-1.277312  , -1.277312  , -1.2908984 , ...,  0.        ,
            0.        ,  0.        ],
          [-1.3044848 , -1.277312  , -1.277312  , ...,  0.        ,
            0.        ,  0.        ],
   

In [ ]:
get_dict == b_dict

False

In [ ]:
for key in get_dict.keys():
    if type(b_dict[key]) == np.ndarray:
        if get_dict[key].shape != b_dict[key].shape:
            print(key, get_dict[key].shape, b_dict[key].shape)
    

points (19639, 5) (17458, 5)


In [ ]:
with torch.no_grad():
    model.load_params_from_file(filename=ckpt_path, logger=logger, to_cpu=to_cpu)
    model.cuda()
    model.eval()
    time.sleep(2)


from detector3d_template.py: def load_params_from_file ******************

2024-06-30 20:15:42,564   INFO  ==> Loading parameters from checkpoint /home/rajeev-gupta/sensyn_ws/src/GD-MAE/data/ckpts/graph_rcnn_voi_kitti.pth to GPU


2024-06-30 20:15:43,113   INFO  ==> Done (loaded 518/518)


from detector3d_template.py: def _load_state_dict ******************


In [ ]:
data_input = get_dict
# data_input = b_dict
torch.cuda.synchronize()
start_time = time.time()
load_data_to_gpu(data_input)
pred_dicts, ret_dict = model(data_input)
torch.cuda.synchronize()
end_time = time.time()
print('Inference Time: ', end_time-start_time)

pred_dicts

from detector3d_template.py: def post_processing ******************
from detector3d_template.py: def generate_recall_record ******************
Inference Time:  2.7641761302948


[{'pred_boxes': tensor([[37.7703, 15.6945, -0.3821,  4.1330,  1.7591,  1.4665,  3.1287]],
         device='cuda:0', grad_fn=<IndexBackward0>),
  'pred_scores': tensor([0.9051], device='cuda:0', grad_fn=<IndexBackward0>),
  'pred_labels': tensor([1], device='cuda:0')}]

In [ ]:
keys = list(b_dict.keys())
keys

['frame_id',
 'calib',
 'points',
 'image',
 'trans_lidar_to_cam',
 'trans_cam_to_img',
 'use_lead_xyz',
 'transformation_2d_list',
 'transformation_2d_params',
 'image_rescale_shape',
 'image_pad_shape',
 'image_shape',
 'batch_size']

In [ ]:
b_dict


{'frame_id': array(['000022'], dtype='<U6'),
 'calib': array([<pcdet.utils.calibration_kitti.Calibration object at 0x7f1f98d70e10>],
       dtype=object),
 'points': array([[ 0.0000e+00,  5.6430e+01,  1.1790e+00,  2.1180e+00,  0.0000e+00],
        [ 0.0000e+00,  6.2003e+01,  1.4930e+00,  2.3060e+00,  0.0000e+00],
        [ 0.0000e+00,  4.7137e+01,  6.2750e+00,  1.8170e+00,  9.2000e-01],
        ...,
        [ 0.0000e+00,  6.3400e+00, -4.1000e-02, -1.6560e+00,  2.7000e-01],
        [ 0.0000e+00,  6.3420e+00, -3.1000e-02, -1.6560e+00,  2.3000e-01],
        [ 0.0000e+00,  6.3380e+00, -1.1000e-02, -1.6550e+00,  1.8000e-01]],
       dtype=float32),
 'image': array([[[[ 5.4155402e-02,  1.4589565e-01,  1.8761617e-01, ...,
            0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
          [-1.2936717e-01,  3.0806570e-04,  4.3419853e-02, ...,
            0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
          [-1.7442521e-01, -4.3367714e-02, -2.0523893e-04, ...,
            0.0000000e+00, 

In [ ]:
type(dataset.__getitem__(0))

def get_fov_flag ******************
point is not None
from def forward ******************
from def mask_points_and_boxes_outside_range ******************
from def shuffle_points ******************
from def calculate_grid_size ******************
from def imrescale ******************
from def imflip ******************
from def imnormalize ******************
from def impad ******************


dict

In [ ]:
info_path = '/media/rajeev-gupta/Drive250/data/kitti/kitti_infos_test.pkl'
with open(info_path, 'rb') as i_file:
    i_dict = pickle.load(i_file)
len(i_dict)

51

In [ ]:
i_dict[0]


{'point_cloud': {'num_features': 4, 'lidar_idx': '000000'},
 'image': {'image_idx': '000000',
  'image_shape': array([ 375, 1242], dtype=int32)},
 'calib': {'P2': array([[7.21537720e+02, 0.00000000e+00, 6.09559326e+02, 4.48572807e+01],
         [0.00000000e+00, 7.21537720e+02, 1.72854004e+02, 2.16379106e-01],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 2.74588400e-03],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  'R0_rect': array([[ 0.9999239 ,  0.00983776, -0.00744505,  0.        ],
         [-0.0098698 ,  0.9999421 , -0.00427846,  0.        ],
         [ 0.00740253,  0.00435161,  0.9999631 ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ]],
        dtype=float32),
  'Tr_velo_to_cam': array([[ 7.53374491e-03, -9.99971390e-01, -6.16602018e-04,
          -4.06976603e-03],
         [ 1.48024904e-02,  7.28073297e-04, -9.99890208e-01,
          -7.63161778e-02],
         [ 9.99862075e-01,  7.52379000e-03,  1.480755

In [ ]:
import os

def process_file(file_path, keyword):
    if not os.path.isfile(file_path):
        print(f"The file {file_path} does not exist.")
        return
    
    # Read the file content
    with open(file_path, 'r') as file:
        lines = file.readlines()
    save_path = file_path[:-3] + '2' + file_path[-3:]
    base_name = os.path.basename(file_path)
    # Find lines containing the keyword
    line_dict = {index: line for index, line in enumerate(lines) if keyword in line}

    # Insert comments below each found line
    for index in sorted(line_dict.keys(), reverse=True):  # Process in reverse order to avoid shifting lines
        line = line_dict[index][:-1]
        i = 0
        while(line[i]==' '):
            i+=1
        j = i
        while(line[j]!='('):
            j+=1
        lines.insert(index + 1, f"{line[:i]}    print('from {base_name}: {line[i:j]} ******************')\n")

    # Write the modified content back to the file
    with open(save_path, 'w') as file:
        file.writelines(lines)

    print(f"Processed file {file_path} successfully.")


In [ ]:
import os

def process_file_del(file_path, keyword):
    if not os.path.isfile(file_path):
        print(f"The file {file_path} does not exist.")
        return
    
    # Read the file content
    with open(file_path, 'r') as file:
        lines = file.readlines()
    save_path = file_path[:-3] + '2' + file_path[-3:]
    # Find lines containing the keyword
    line_dict = {index: line for index, line in enumerate(lines) if keyword in line}

    # Insert comments below each found line
    for index in sorted(line_dict.keys(), reverse=True):  # Process in reverse order to avoid shifting lines
        del lines[index]
    # Write the modified content back to the file
    with open(save_path, 'w') as file:
        file.writelines(lines)

    print(f"Processed file {file_path} successfully.")


In [ ]:

# file_path = '/home/rajeev-gupta/sensyn_ws/src/GD-MAE/pcdet/datasets/kitti/kitti_dataset.py'  # Replace with your file path
# keyword = '******************\')'
# process_file_del(file_path, keyword)


Processed file /home/rajeev-gupta/sensyn_ws/src/GD-MAE/pcdet/datasets/kitti/kitti_dataset.py successfully.


In [ ]:

# file_path = '/home/rajeev-gupta/sensyn_ws/src/GD-MAE/pcdet/datasets/dataset.py'  # Replace with your file path
# keyword = '******************\')'
# process_file_del(file_path, keyword)


Processed file /home/rajeev-gupta/sensyn_ws/src/GD-MAE/pcdet/datasets/dataset.py successfully.


In [ ]:
# file_path = '/home/rajeev-gupta/sensyn_ws/src/GD-MAE/pcdet/models/detectors/detector3d_template.py'  # Replace with your file path
# keyword = '******************\')'
# process_file_del(file_path, keyword)


Processed file /home/rajeev-gupta/sensyn_ws/src/GD-MAE/pcdet/models/detectors/detector3d_template.py successfully.


In [ ]:
file_path = '/home/rajeev-gupta/sensyn_ws/src/GD-MAE/pcdet/datasets/processor/data_processor.py'  # Replace with your file path
keyword = '******************\')'
process_file_del(file_path, keyword)


Processed file /home/rajeev-gupta/sensyn_ws/src/GD-MAE/pcdet/datasets/processor/data_processor.py successfully.
